**stage3_model_eval**. This notebook evaluates the trained model

**Edit**<br/>

**TODO**<br/>

# Import packages and get authenticated

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [ ]:
import numpy as np
import copy
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = "{:,.6f}".format

from tqdm import tqdm_notebook as tqdm
from IPython.display import display
from PIL import Image
import os
import sys
sys.path.append('/content/drive/My Drive/中研院/repo/')
# sys.path.append('~/project_FDDAT/repo/')
sys.path.append('../') # add this line so Data and data are visible in this file
from os.path import expanduser
home = expanduser("~")

from falldetect.utilities import *
from falldetect.models import *
from falldetect.dataset_util import *
from falldetect.training_util import *
from falldetect.eval_util import *

import time
import datetime
from datetime import datetime
import json
import argparse

# Plotting
# checklist 1: comment inline, uncomment Agg
%matplotlib inline
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import pyplot
matplotlib.rc( 'savefig', facecolor = 'white' )

from sklearn.decomposition import PCA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Get user inputs
In ipython notebook, these are hardcoded. In production python code, use parsers to provide these inputs

In [ ]:
parser = argparse.ArgumentParser(description='FD_DAT')
parser.add_argument('--input_dir', metavar='input_dir', help='input_dir',
                    default='../')
parser.add_argument('--stage1_folder', metavar='stage1_folder', help='stage1_folder',
                    default='../')
parser.add_argument('--stage2_folder', metavar='stage2_folder', help='stage2_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')
parser.add_argument('--tasks_list', metavar='tasks_list', help='a list of all tasks',
                    default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
parser.add_argument('--variable_name', metavar='variable_name', help='key in training_params to be displayed on plot',
                    default='HP_name')
parser.add_argument('--debug_F1', metavar='debug_F1', help='debug F1',
                    default='False')



# parser.add_argument('--src_names', metavar='src_names', help='a list of src_names',
#                     default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
# parser.add_argument('--tgt_names', metavar='tgt_names', help='a list of tgt_names',
#                     default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')

# checklist 2: comment first line, uncomment second line seizures_FN

# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_diffCV',
#                           '--output_folder', '../../data_mic/stage3/test',


# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_WithoutNormal_18hz_5fold_UPFall_UMAFall_cross-config_diffCV',
# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_diffCV_earlystop',
# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/test',


# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_HPsearch',
args = parser.parse_args(['--input_dir', '../../Data/{}/ImpactWindow_Resample/18hz/{}/',
                          '--stage1_folder', '../../data_mic/stage1/preprocessed_18hz_5fold',
                          '--stage2_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_HPsearch', 
                          '--output_folder', '../../data_mic/stage3/test',
                          '--training_params_file', 'training_params_list_HPsearch.json',
#                           '--training_params_file', 'training_params_list_HPsearch.json',
#                           '--tasks_list', 'UMAFall_chest-UPFall_neck UMAFall_wrist-UPFall_wrist UMAFall_waist-UPFall_belt UMAFall_leg-UPFall_rightpocket UMAFall_ankle-UPFall_ankle',
                          '--tasks_list', 'UMAFall_chest-UPFall_neck UMAFall_leg-UPFall_rightpocket',
                          '--variable_name', 'HP_name',])
#                           '--debug_F1', 'True',])

#                           '--src_names', 'UPFall_neck UPFall_wrist UPFall_belt UPFall_rightpocket UPFall_ankle',
#                           '--tgt_names', 'UMAFall_chest UMAFall_wrist UMAFall_waist UMAFall_leg UMAFall_ankle'])

# args = parser.parse_args()

In [ ]:
# project_FDDAT/data_mic/stage2/modeloutput_18hz_5fold_UMAFall_cross-pos_loss_earlystop_HPsearch/UMAFall_ankle_UMAFall_chest/HP_24/source/rep1/class_out_diagnosis_CV4_epoch14.png

In [ ]:
home_dir = home+'/project_FDDAT/'
input_dir = args.input_dir
stage1_folder = args.stage1_folder
stage2_folder = args.stage2_folder
output_folder = args.output_folder
training_params_file = args.training_params_file

tasks_list = []
for item in args.tasks_list.split(' '):
    tasks_list.append((item.split('-')[0], item.split('-')[1]))
    
# src_domains = args.src_names.split(' ')
# tgt_domains = args.tgt_names.split(' ')
variable_name = args.variable_name

# if args.debug_F1=='True':
#     debug_F1 = True
# else:
#     debug_F1 = False


inputdir_stage1 = stage1_folder + '/'
inputdir_stage2 = stage2_folder + '/'
outputdir = output_folder + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [ ]:
outputdir

In [ ]:
# with open('../stage2/'+training_params_file) as json_file:
#     training_params_list = json.load(json_file)

# # TODO: need to fix once training_params_list is fixed
# # training_params_list.pop(-2)
# training_params_list

In [ ]:
# training_mode = 'dann'

# src_name = 'UMAFall_ankle'
# tgt_name = 'UPFall_ankle'

# i_rep = 0
# i_CV = 0
# stage2_outdir = home+'/project_FDDAT/data_mic/stage2/test/{}_{}/HP_fixed/{}/rep{}/tgt_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, training_mode, i_rep, i_CV)

# tgt_class_output = np.load(stage2_outdir, allow_pickle=True)['data']
# tgt_class_sigmoid = tgt_class_output[:,0]
# tgt_DataNameList_idx = tgt_class_output[:,1]

In [ ]:
# tgt_DataNameList_idx

In [ ]:
# HP_name = 'HP_22'
# inputdir_stage2+'{}_{}/{}/rep{}/src_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, HP_name, i_rep, i_CV)

In [ ]:
inputdir_stage2.format(src_name.split('_')[0],tgt_name.split('_')[0])
# src_name

In [ ]:
training_mode = 'dann'

def get_df_DataNameList(data_name, input_dir):
    sensor_loc = data_name.split('_')[1]
    dataset_name = data_name.split('_')[0]

    DataNameList_inputdir = input_dir+'IP_{}_DataNameList_{}.csv'
    DataNameList_inputdir = DataNameList_inputdir.format(dataset_name, sensor_loc, dataset_name, sensor_loc)

    impact_inputdir = input_dir.format(dataset_name, sensor_loc)

    df_DataNameList = pd.read_csv(DataNameList_inputdir)
    return df_DataNameList


def get_model_output(src_name, tgt_name, training_mode, i_rep, i_CV):
    HP_name = 'HP_0'
    
#     model_outputdir = inputdir_stage2.format(src_name.split('_')[0],tgt_name.split('_')[0])
#     stage2_outdir = home+'/project_FDDAT/data_mic/stage2/test/{}_{}/HP_fixed/{}/rep{}/src_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, training_mode, i_rep, i_CV)
    stage2_outdir = inputdir_stage2+'{}_{}/{}/{}/rep{}/src_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, HP_name, training_mode, i_rep, i_CV)
    model_output_src = np.load(stage2_outdir, allow_pickle=True)['data']

#     stage2_outdir = home+'/project_FDDAT/data_mic/stage2/test/{}_{}/HP_fixed/{}/rep{}/tgt_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, training_mode, i_rep, i_CV)
    stage2_outdir = inputdir_stage2+'{}_{}/{}/{}/rep{}/tgt_class_sigmoid_CV{}.npz'.format(src_name, tgt_name, HP_name, training_mode, i_rep, i_CV)
    model_output_tgt = np.load(stage2_outdir, allow_pickle=True)['data']
    
    return model_output_src, model_output_tgt

def df_DataNameList_expand(i_rep, i_CV, df_DataNameList, DataNameList_idx, class_sigmoid):
    col_name = 'rep{}_CV{}'.format(i_rep, i_CV)
    df_DataNameList[col_name] = ''
    df_DataNameList.loc[DataNameList_idx, [col_name]] = class_sigmoid
    return df_DataNameList



In [ ]:


src_name = tasks_list[0][0]
tgt_name = tasks_list[0][1]

i_rep = 0
i_CV = 0

df_DataNameList_src = get_df_DataNameList(src_name, input_dir)
df_DataNameList_tgt = get_df_DataNameList(tgt_name, input_dir)

for i_rep in range(5):
    for i_CV in range(5):
        model_output_src, model_output_tgt = get_model_output(src_name, tgt_name, training_mode, i_rep, i_CV)
        class_sigmoid_src = model_output_src[:,0]
        DataNameList_idx_src = model_output_src[:,1]
        class_sigmoid_tgt = model_output_tgt[:,0]
        DataNameList_idx_tgt = model_output_tgt[:,1]
        
        df_DataNameList_src = df_DataNameList_expand(i_rep, i_CV, df_DataNameList_src, DataNameList_idx_src, class_sigmoid_src)
        df_DataNameList_tgt = df_DataNameList_expand(i_rep, i_CV, df_DataNameList_tgt, DataNameList_idx_tgt, class_sigmoid_tgt)



In [ ]:
display(df_DataNameList_src)

df_DataNameList_src.to_csv("df_DataNameList_src.csv", index=False)



In [ ]:
src_inputdir = inputdir + '{}/{}/rep{}/'.format(src_name.split('_')[0], src_name.split('_')[1], i_rep)
tgt_inputdir = inputdir + '{}/{}/rep{}/'.format(tgt_name.split('_')[0], tgt_name.split('_')[1], i_rep)



# for i_CV in range(CV_n):
i_CV = 0
print('------------------------------Working on i_CV {}------------------------------'.format(i_CV))
# 1. prepare dataset

batch_size = training_params['batch_size']
learning_rate = training_params['learning_rate']
training_params['use_WeightedRandomSampler'] = False
src_train_loader, src_val_loader, src_train_loader_eval = get_data_loader(src_inputdir, i_CV, batch_size, learning_rate, training_params)
tgt_train_loader, tgt_val_loader, tgt_train_loader_eval = get_data_loader(tgt_inputdir, i_CV, batch_size, learning_rate, training_params)

In [ ]:
# tgt_val_loader.dataset.labels.shape

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=80)
ax = fig.add_subplot(1, 1, 1)
# ax.plot(tgt_class_sigmoid[:,0],'.b', label='src_domain_sigmoid', markersize=3)


# ax3 = fig.add_subplot(4, 2, ax_idx[2])
ax.plot(tgt_class_sigmoid[:,0],'.b', label='tgt_class_sigmoid', markersize=3)
ax.plot(tgt_class_sigmoid[:,0].round(),'b', alpha=0.5, label='tgt_class_decision')
ax.plot(tgt_val_loader.dataset.labels,'r', alpha=0.5, label='tgt_class_labels')
# ax3.set_title('tgt_class_sigmoid (adl=0, fall=1)')
ax.legend(loc='upper right')

In [ ]:
tgt_class_sigmoid[:,1]